In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
legacy = pd.read_csv('/content/male_players (legacy).csv')

<ipython-input-3-9b4e76398638>:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  legacy = pd.read_csv('/content/male_players (legacy).csv')


In [4]:
legacy.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [5]:
legacy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48739 entries, 0 to 48738
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 40.9+ MB


In [6]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

def preprocessing(legacy):
    # Checking for missing values
    numeric_columns = legacy.select_dtypes(include=['float64', 'int64'])
    missing_values = numeric_columns.isnull().sum()

    # Replacing missing values with the mean of its columns
    legacy[numeric_columns.columns] = legacy[numeric_columns.columns].fillna(numeric_columns.mean())

    # Dropping specified columns
    columns_to_drop = [
          "player_positions", "league_name", "club_name",
        "dob", "player_url", "long_name", "short_name", "club_team_id",
        "player_face_url", "real_face", "lcm", "cm", "rcm",
        "lf", "rf", "ldm", "cdm", "rdm", "rwb", "lb", "lcb", "cb", "rcb",
        "cf","lw","rw","lam","ram","cam","rm","lm","st","ls","rs","lwb","rb"
    ]
    legacy = legacy.drop(columns=columns_to_drop)

    # Filtering columns based on missing values threshold
    L = []
    L_less = []
    for i in legacy.columns:
        if legacy[i].isnull().sum() < (0.3 * legacy.shape[0]):
            L.append(i)
        else:
            L_less.append(i)

    legacy = legacy[L]

    # Separating numeric and non-numeric features
    numeric_columns = legacy.select_dtypes(exclude=['object']).columns
    non_numeric = legacy.select_dtypes(include=['object']).columns

    cat_data = legacy[non_numeric]
    numeric_data = legacy[numeric_columns]

    # Imputing the numeric data with the median
    imp = SimpleImputer(strategy='median')
    a = imp.fit_transform(numeric_data)
    numeric_data_impute = pd.DataFrame(a, columns=numeric_columns)

    # Filling missing values in categorical data using forward fill
    cat_data_processed = cat_data.fillna(method='ffill')
    cat_data_processed = pd.DataFrame(cat_data_processed, columns=cat_data.columns)

    # Encoding the categorical data
    label_encoders = {}
    for column in cat_data_processed.columns:
        cat_data_processed[column] = cat_data_processed[column].astype(str)
        label_encoders[column] = LabelEncoder()
        cat_data_processed[column] = label_encoders[column].fit_transform(cat_data_processed[column])

    # Joining numeric and processed categorical data
    legacy = pd.concat([numeric_data_impute, cat_data_processed], axis=1)

    return legacy

In [7]:
processed_legacy =  preprocessing(legacy)

In [8]:
processed_legacy

,player_id,fifa_version,fifa_update,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,goalkeeping_reflexes,goalkeeping_speed,fifa_update_date,club_position,club_joined_date,nationality_name,preferred_foot,work_rate,body_type,gk
0,158023.0,15.0,2.0,93.0,95.0,100500000.0,550000.0,27.0,169.0,67.0,...,8.0,43.278305,0,3,43,4,0,7,3,22
1,20801.0,15.0,2.0,92.0,92.0,79000000.0,375000.0,29.0,185.0,80.0,...,11.0,43.278305,0,14,236,126,1,1,5,26
2,9014.0,15.0,2.0,90.0,90.0,54500000.0,275000.0,30.0,180.0,80.0,...,15.0,43.278305,0,28,254,109,0,1,4,18
3,41236.0,15.0,2.0,90.0,90.0,52500000.0,275000.0,32.0,195.0,95.0,...,12.0,43.278305,0,27,612,148,1,7,5,30
4,167495.0,15.0,2.0,90.0,90.0,63500000.0,300000.0,28.0,193.0,92.0,...,86.0,60.000000,0,5,449,60,1,8,5,174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48734,216226.0,17.0,2.0,56.0,58.0,120000.0,2000.0,23.0,180.0,76.0,...,14.0,43.278305,2,19,839,48,0,5,7,28
48735,216299.0,17.0,2.0,56.0,70.0,200000.0,9000.0,20.0,180.0,76.0,...,12.0,43.278305,2,21,839,48,1,8,7,16
48736,216555.0,17.0,2.0,56.0,64.0,150000.0,500.0,22.0,181.0,76.0,...,16.0,43.278305,2,19,1689,129,1,8,1,24
48737,216563.0,17.0,2.0,56.0,70.0,230000.0,2000.0,21.0,180.0,68.0,...,15.0,43.278305,2,28,873,163,1,7,1,24


In [9]:
#question 2
corr_matrix = processed_legacy.corr()
corr_overall = corr_matrix['overall'].sort_values(ascending=False)

top_features = 10
features = corr_overall.head(top_features)
print(features)

overall                     1.000000
movement_reactions          0.806883
potential                   0.777970
wage_eur                    0.676786
passing                     0.599985
value_eur                   0.591997
dribbling                   0.521610
international_reputation    0.511567
attacking_short_passing     0.467187
skill_long_passing          0.454085
Name: overall, dtype: float64


In [10]:
#question 3
selected_columns=["overall","movement_reactions","mentality_composure","potential","wage_eur","passing","attacking_short_passing","attacking_short_passing","value_eur","dribbling"]
#droping non_selected columns
non_selected_columns = [col for col in processed_legacy.columns if col not in selected_columns]

processed_legacy.drop(non_selected_columns, axis=1, inplace=True)
processed_legacy

,overall,potential,value_eur,wage_eur,passing,dribbling,attacking_short_passing,movement_reactions,mentality_composure
0,93.0,95.0,100500000.0,550000.0,86.000000,96.000000,89.0,94.0,57.276924
1,92.0,92.0,79000000.0,375000.0,81.000000,91.000000,82.0,90.0,57.276924
2,90.0,90.0,54500000.0,275000.0,83.000000,92.000000,86.0,89.0,57.276924
3,90.0,90.0,52500000.0,275000.0,81.000000,86.000000,84.0,85.0,57.276924
4,90.0,90.0,63500000.0,300000.0,56.786659,61.690949,42.0,89.0,57.276924
...,...,...,...,...,...,...,...,...,...
48734,56.0,58.0,120000.0,2000.0,53.000000,57.000000,60.0,51.0,51.000000
48735,56.0,70.0,200000.0,9000.0,53.000000,56.000000,62.0,57.0,58.000000
48736,56.0,64.0,150000.0,500.0,55.000000,57.000000,58.0,60.0,55.000000
48737,56.0,70.0,230000.0,2000.0,57.000000,58.000000,61.0,43.0,59.000000


In [11]:
#Question 3
#training the model
#x=processed_legacy
#y=processed_legacy['overall']
y = processed_legacy['overall']
processed_legacy = processed_legacy.drop(columns=['overall'])
x = processed_legacy

In [12]:
#scaling x
from sklearn.preprocessing import StandardScaler
#creating an instance of standscaler
scaler=StandardScaler()
scaled=scaler.fit_transform(x)

In [13]:
#splitting the dataset into training and testing sets
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest=train_test_split(x,y,test_size = 0.2,random_state =42)


In [14]:
Xtrain.shape

(38991, 8)

In [15]:
Xtest.shape

(9748, 8)

In [16]:
# pickling the model
#creating the scaler model
import pickle
pickle_out = open("scaler.pkl", "wb")
pickle.dump(scaled, pickle_out)
pickle_out.close()


In [17]:
!pip install xgboost


In [18]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import  xgboost as xgb
from sklearn.metrics import mean_absolute_error
#Random Forest
rf=RandomForestRegressor()
rf.fit(Xtrain,Ytrain)

#gradientboost regressor
grad_boost=GradientBoostingRegressor(max_depth=2,n_estimators=3,learning_rate=1.0)
grad_boost.fit(Xtrain,Ytrain)
#XGBoost
xgb_reg=xgb.XGBRegressor()
xgb_reg.fit(Xtrain,Ytrain)

#Finding the mean score for each model to get each model's performance
#then hyperparameter tune
rf_score=cross_val_score(rf, Xtest, Ytest,cv=5)
print("The Random Forest Regressor score is :", rf_score.mean())

#displaying the gradientboost score
gradb_score=cross_val_score(grad_boost, Xtest, Ytest,cv=5)
print("The Gradient Boosting Regressor score is :", gradb_score.mean())

xgb_reg_score=cross_val_score(rf, Xtest, Ytest,cv=5)
print("The XGBoost Regressor score is :", xgb_reg_score.mean())

The Random Forest Regressor score is : 0.9610915025457712
The Gradient Boosting Regressor score is : 0.8537391072628078
The XGBoost Regressor score is : 0.961108261949739


In [19]:

#Grid search and cross-validation
#ensemble Random forest regressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import  xgboost as xgb
from sklearn.metrics import mean_absolute_error

#Initialize Random Forest Regressor
rf = RandomForestRegressor()
#hyperparameters and their possible values for the grid search
param_grid = {
    'n_estimators': [60, 100, 200],
    'max_depth': [None, 10, 20, 30],
}
#grid search with cross-validation
grid = GridSearchCV(estimator=rf, param_grid=param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=42))

#grid search fit legacy (training data)
grid.fit(Xtrain, Ytrain)

#obtain the best model with the best hyperparameters
best_rf = grid.best_estimator_

# Predicting on the test data using the best model
y_predict = best_rf.predict(Xtest)
mae = mean_absolute_error(Ytest, y_predict)
print("Mean Absolute Error for the Best Random Forest Model:", mae)

Mean Absolute Error for the Best Random Forest Model: 0.7639262444783533


In [20]:
#gradientboosting regressor
grad_b = GradientBoostingRegressor()

#grid search with cross- validation for gradientBoosting
grid_grad_boost = GridSearchCV(estimator=grad_b, param_grid=param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=42))
#grid search fit to the training data
grid_grad_boost.fit(Xtrain, Ytrain)

# obtain the best model with the best hyperparameters
best_grad_boost = grid_grad_boost.best_estimator_

#data fitting with the model
best_grad_boost.fit(Xtrain, Ytrain)

# Predicting on the test data
y_predict = best_grad_boost.predict(Xtest)

# Mean Absolute Error (MAE) value
mae = mean_absolute_error(Ytest, y_predict)
print("Mean Absolute Error for the Best Gradient Boosting Model:", mae)

Mean Absolute Error for the Best Gradient Boosting Model: 0.7993158478448086


In [21]:
#XGBoost
xgb_reg = xgb.XGBRegressor()
param = {
    'n_estimators': [60, 150, 220],
    'max_depth': [10, 15, 20] ,
    }
#grid search with cross-validation for the XGBoost
grid_xgb = GridSearchCV(estimator=xgb_reg, param_grid=param, cv=KFold(n_splits=5, shuffle=True, random_state=42))

#grid search fit to the training data
grid_xgb.fit(Xtrain, Ytrain)

#obtaining the best model with the best hyperparameters
best_xgb = grid_xgb.best_estimator_

#Fit the best model to the training data
best_xgb.fit(Xtrain, Ytrain)

#Predicting on the test data
y_predict = best_xgb.predict(Xtest)

#Mean Absolute Error (MAE) value
mae = mean_absolute_error(Ytest, y_predict)
print("Mean Absolute Error for the Best XGB Regressor Model:", mae)

Mean Absolute Error for the Best XGB Regressor Model: 0.8245401601591803


In [22]:
from sklearn.ensemble import VotingRegressor
voting_reg = VotingRegressor(estimators=[
    ('RandomForest', best_rf),
    ('GradientBoosting', best_grad_boost),
    ('XGBoost', best_xgb)
])
# Fitting & training data
voting_reg.fit(Xtrain, Ytrain)
# test data predictions using the ensemble model
ensemble_pred= voting_reg.predict(Xtest)

# Evaluating using mean squared error(mse)
from sklearn.metrics import mean_squared_error

# Evaluating using mean absolute error (mae)
mae = mean_absolute_error(ensemble_pred, Ytest)
print("Mean Absolute Error for Ensemble:", mae)

Mean Absolute Error for Ensemble: 0.7724257854494616


In [23]:
import pickle
pickle_out = open("voting_reg.pkl", "wb")
pickle.dump(voting_reg, pickle_out)
pickle_out.close()

Test

In [24]:
players_22 = pd.read_csv('/content/players_22-1.csv')

<ipython-input-24-d580a52fc77f>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_22 = pd.read_csv('/content/players_22-1.csv')


In [25]:
players_22.head()


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [26]:
processed_players_22 =  preprocessing(players_22)

In [27]:
processed_players_22

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_jersey_number,...,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,gk,club_logo_url,club_flag_url,nation_flag_url
0,158023.0,93.0,93.0,78000000.0,320000.0,34.0,170.0,72.0,1.0,30.0,...,25,1808,6,0,7,9,18,663,19,7
1,188545.0,92.0,92.0,119500000.0,270000.0,32.0,185.0,81.0,1.0,9.0,...,27,210,123,1,2,9,18,500,14,125
2,20801.0,91.0,91.0,45000000.0,270000.0,36.0,187.0,83.0,1.0,7.0,...,27,1825,124,1,1,9,20,68,20,128
3,190871.0,91.0,91.0,129000000.0,270000.0,29.0,175.0,68.0,1.0,10.0,...,14,652,20,1,2,9,20,663,19,20
4,192985.0,91.0,91.0,125500000.0,350000.0,30.0,181.0,70.0,1.0,17.0,...,19,353,13,1,0,9,22,1,20,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962.0,47.0,52.0,70000.0,1000.0,22.0,180.0,64.0,1.0,36.0,...,28,1682,30,1,8,1,9,251,10,32
19235,262040.0,47.0,59.0,110000.0,500.0,19.0,175.0,70.0,1.0,27.0,...,21,1645,126,1,8,1,7,581,26,77
19236,262760.0,47.0,55.0,100000.0,500.0,21.0,178.0,72.0,1.0,31.0,...,21,1755,126,1,8,1,3,161,26,77
19237,262820.0,47.0,60.0,110000.0,500.0,19.0,173.0,66.0,1.0,12.0,...,21,1755,126,1,8,1,9,161,26,77


In [37]:
# test_sub_columns = ["movement_reactions","mentality_composure","potential","wage_eur","passing","attacking_short_passing","value_eur","dribbling"]
# #top feature subsets
# top_feat = players_22[test_sub_columns]
# #display top feat
# top_feat

,movement_reactions,mentality_composure,potential,wage_eur,passing,attacking_short_passing,value_eur,dribbling
0,94,96,93,320000.0,91.0,91,78000000.0,95.0
1,93,88,92,270000.0,79.0,85,119500000.0,86.0
2,94,95,91,270000.0,80.0,80,45000000.0,88.0
3,89,93,91,270000.0,86.0,86,129000000.0,94.0
4,91,89,91,350000.0,93.0,94,125500000.0,88.0
...,...,...,...,...,...,...,...,...
19234,53,37,52,1000.0,46.0,50,70000.0,48.0
19235,49,47,59,500.0,50.0,51,110000.0,46.0
19236,46,36,55,500.0,45.0,49,100000.0,49.0
19237,48,47,60,500.0,36.0,38,110000.0,48.0


In [42]:
sc = StandardScaler()
scaled = sc.fit_transform(processed_players_22)
top_feat = pd.DataFrame(scaled,columns = processed_players_22.columns)
top_feat

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_jersey_number,...,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,gk,club_logo_url,club_flag_url,nation_flag_url
0,-2.716263,3.957501,3.601780,9.889601,15.998022,1.851089,-1.646467,-0.416315,-0.4746,0.506404,...,0.485315,1.112439,-1.519483,-1.792891,0.458733,2.997879,0.068706,1.534514,-0.319645,-1.404437
1,-1.587450,3.812154,3.437470,15.350958,13.425844,1.429869,0.539166,0.856805,-0.4746,-0.668061,...,0.727947,-2.310810,0.894005,0.557758,-1.265143,2.997879,0.068706,0.729267,-0.715283,1.191834
2,-7.791226,3.666806,3.273160,5.546836,13.425844,2.272309,0.830584,1.139720,-0.4746,-0.779914,...,0.727947,1.148857,0.914633,0.557758,-1.609919,2.997879,0.184414,-1.404887,-0.240517,1.257841
3,-1.501426,3.666806,3.273160,16.601147,13.425844,0.798039,-0.917923,-0.982145,-0.4746,-0.612134,...,-0.849162,-1.363954,-1.230690,0.557758,-1.265143,2.997879,0.184414,1.534514,-0.319645,-1.118407
4,-1.423243,3.666806,3.273160,16.140551,17.541329,1.008649,-0.043669,-0.699230,-0.4746,-0.220646,...,-0.242582,-2.004474,-1.375087,0.557758,-1.954694,2.997879,0.300122,-1.735878,-0.240517,-1.272423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,1.127775,-2.728494,-3.134932,-0.365905,-0.412474,-0.676231,-0.189378,-1.547976,-0.4746,0.841965,...,0.849263,0.842521,-1.024409,0.557758,0.803508,-1.326646,-0.451978,-0.500835,-1.031794,-0.854379
19235,1.130659,-2.728494,-1.984762,-0.360641,-0.438196,-1.308061,-0.917923,-0.699230,-0.4746,0.338623,...,0.000050,0.763259,0.955889,0.557758,0.803508,-1.326646,-0.567685,1.129421,0.234249,0.135724
19236,1.157287,-2.728494,-2.642002,-0.361957,-0.438196,-0.886841,-0.480796,-0.416315,-0.4746,0.562331,...,0.000050,0.998902,0.955889,0.557758,0.803508,-1.326646,-0.799101,-0.945451,0.234249,0.135724
19237,1.159506,-2.728494,-1.820452,-0.360641,-0.438196,-1.308061,-1.209340,-1.265061,-0.4746,-0.500280,...,0.000050,0.998902,0.955889,0.557758,0.803508,-1.326646,-0.451978,-0.945451,0.234249,0.135724


In [43]:
X_test = top_feat
X_test


,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_jersey_number,...,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,gk,club_logo_url,club_flag_url,nation_flag_url
0,-2.716263,3.957501,3.601780,9.889601,15.998022,1.851089,-1.646467,-0.416315,-0.4746,0.506404,...,0.485315,1.112439,-1.519483,-1.792891,0.458733,2.997879,0.068706,1.534514,-0.319645,-1.404437
1,-1.587450,3.812154,3.437470,15.350958,13.425844,1.429869,0.539166,0.856805,-0.4746,-0.668061,...,0.727947,-2.310810,0.894005,0.557758,-1.265143,2.997879,0.068706,0.729267,-0.715283,1.191834
2,-7.791226,3.666806,3.273160,5.546836,13.425844,2.272309,0.830584,1.139720,-0.4746,-0.779914,...,0.727947,1.148857,0.914633,0.557758,-1.609919,2.997879,0.184414,-1.404887,-0.240517,1.257841
3,-1.501426,3.666806,3.273160,16.601147,13.425844,0.798039,-0.917923,-0.982145,-0.4746,-0.612134,...,-0.849162,-1.363954,-1.230690,0.557758,-1.265143,2.997879,0.184414,1.534514,-0.319645,-1.118407
4,-1.423243,3.666806,3.273160,16.140551,17.541329,1.008649,-0.043669,-0.699230,-0.4746,-0.220646,...,-0.242582,-2.004474,-1.375087,0.557758,-1.954694,2.997879,0.300122,-1.735878,-0.240517,-1.272423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,1.127775,-2.728494,-3.134932,-0.365905,-0.412474,-0.676231,-0.189378,-1.547976,-0.4746,0.841965,...,0.849263,0.842521,-1.024409,0.557758,0.803508,-1.326646,-0.451978,-0.500835,-1.031794,-0.854379
19235,1.130659,-2.728494,-1.984762,-0.360641,-0.438196,-1.308061,-0.917923,-0.699230,-0.4746,0.338623,...,0.000050,0.763259,0.955889,0.557758,0.803508,-1.326646,-0.567685,1.129421,0.234249,0.135724
19236,1.157287,-2.728494,-2.642002,-0.361957,-0.438196,-0.886841,-0.480796,-0.416315,-0.4746,0.562331,...,0.000050,0.998902,0.955889,0.557758,0.803508,-1.326646,-0.799101,-0.945451,0.234249,0.135724
19237,1.159506,-2.728494,-1.820452,-0.360641,-0.438196,-1.308061,-1.209340,-1.265061,-0.4746,-0.500280,...,0.000050,0.998902,0.955889,0.557758,0.803508,-1.326646,-0.451978,-0.945451,0.234249,0.135724
